In [1]:
# ensure packages are installed, uninstall normal tensorflow and ensure gpu can be used
!pip uninstall tensorflow -y 
!pip install pandas tensorflow-gpu numpy chess

Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 394.3MB 40kB/s 
     |████████████████████████████████| 143kB 59.9MB/s 


In [16]:
import getpass  # For hiding token input

# input forms for GitHub username, email address and token
GIT_USERNAME = input("\nEnter GitHub user name: ").strip()
GIT_EMAIL = input("\nEnter GitHub email address: ").strip()
GIT_TOKEN = getpass.getpass("\nEnter GitHub token: ").strip()


Enter GitHub user name: larsaars

Enter GitHub email address: lurzapps@gmail.com

Enter GitHub token: ··········


In [20]:
import os

%cd /content/

# Clean sample_data
sd = 'sample_data'
if os.path.exists(sd):
  print(f"removing {sd}!")
  !rm -rf {sd}

# GitHub repository name
GIT_REPOSITORY = 'chess_heuristics_deep_learning'

# Path to GitHub repository
GIT_PATH = 'https://' + GIT_TOKEN + '@github.com/' + GIT_USERNAME + '/' + GIT_REPOSITORY + '.git'
print(GIT_PATH)

# Clone or pull git repository
if not os.path.exists(GIT_REPOSITORY):
  # Clone the github repository
  !git clone {GIT_PATH}
else:
  # else pull the github repository
  %cd /content/{GIT_REPOSITORY}
  !git pull {GIT_PATH}

%cd /content/{GIT_REPOSITORY}
print("\nDone.")

/content
https://972ed18d57df070d7a7175102975ce3c743b1fde@github.com/larsaars/chess_heuristics_deep_learning.git
/content/chess_heuristics_deep_learning
fatal: not a git repository (or any of the parent directories): .git
/content/chess_heuristics_deep_learning

Done.


In [4]:
# upload the dataset to colab, the path is defined here
dataset_path = 'games.csv'

In [5]:
# make imports
import pandas as pd
import numpy as np
import chess
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf

from tensorflow.keras.layers import BatchNormalization, Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential

In [6]:
# ensure tensorflow is imported properly
print(tf.__version__)
tf.config.list_physical_devices('GPU')

2.4.1


[]

In [7]:
# utility functions for chess
chess_dict = {
    'p': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'P': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    'n': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'N': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    'b': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'B': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    'r': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    'R': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    'q': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    'Q': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    'k': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    'K': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    '.': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
}


def make_matrix(board):
    pgn = board.epd()
    foo = []
    pieces = pgn.split(" ", 1)[0]
    rows = pieces.split("/")
    for row in rows:
        foo2 = []
        for thing in row:
            if thing.isdigit():
                for i in range(0, int(thing)):
                    foo2.append('.')
            else:
                foo2.append(thing)
        foo.append(foo2)
    return foo


def translate(matrix):
    rows = []
    for row in matrix:
        terms = []
        for term in row:
            terms.append(chess_dict[term])
        rows.append(terms)
    return rows

In [8]:
# create the model and compile
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=1, activation='relu', input_shape=(8, 8, 12)))
model.add(MaxPooling2D())
model.add(Conv2D(filters=24, kernel_size=1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(filters=10, kernel_size=1, activation='relu'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(1, activation='tanh'))

model.compile(optimizer='Nadam', loss='mse')

In [9]:
# read the pandas dataframe and create X and y
df = pd.read_csv(dataset_path, sep=';')
df = df[df['winner'] != 'draw']
df = df[df['victory_status'] != 'outoftime']
moves = df['moves'].values
winner = df['winner'].values
X = []
y = []

FileNotFoundError: ignored

In [ ]:

# create dataset
index = 0
for game in moves:
    all_moves = game.split()
    total_moves = len(all_moves)
    if winner[index] == 'black':
        game_winner = -1
    else:
        game_winner = 1

    board = chess.Board()
    for i in range(len(all_moves)):
        board.push_san(all_moves[i])
        value = game_winner * (i / total_moves)
        matrix = make_matrix(board.copy())
        rows = translate(matrix)
        X.append([rows])
        y.append(value)

    index += 1

In [ ]:
# redefine X and y as numpy arrays (and reshape X)
X = np.array(X).reshape((len(X), 8, 8, 12))
y = np.array(y)

In [ ]:
# start training
# model file name
h5 = 'model.h5'

# create callbacks
checkpoint = keras.callbacks.ModelCheckpoint(h5,
                                             monitor='loss',
                                             verbose=1,
                                             save_best_only=True,
                                             save_weights_only=True,
                                             mode='auto',
                                             save_freq=1)
es = keras.callbacks.EarlyStopping(monitor='loss', mode='min', verbose=1, patience=500)

# not a 1000 epochs since we want the repository to be automatically pushed back once done with the epochs
model.fit(X, y, epochs=1, verbose=2, callbacks=[checkpoint, es])

In [ ]:
# push repository (updated model etc.) back to github
%cd /content/{GIT_REPOSITORY}
"""
git add -A stages all changes
git add . stages new files and modifications, without deletions
git add -u stages modifications and deletions, without new files
"""
!git add -A

!git config --global user.email {GIT_EMAIL}
!git config --global user.name {GIT_USERNAME}
!git commit -m "Automatically commit and push through Google Colab!"

!git push origin main